# Model recommendation with lighfm

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation
import json

### Defining variables

In [ ]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:

TEST_PERCENTAGE = 0.20
LEARNING_RATE = 0.1
NUM_EPOCHS = 80
NUM_COMPONENTS = 10
NUM_THREADS = 3
ALPHA_REG_L2 = 1e-3
MAX_SAMPLED = 10
SEED = 42

### Retrieve data

In [4]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [ ]:
df_merged = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_merged.drop(columns=["Unnamed: 0"],inplace=True)
df_merged

,Unnamed: 0,userId,history,userType,score
0,0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...,...
6349891,6349891,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,7a349b09-badc-40a9-a194-83d959aeb50c,Non-Logged,1.94055
6349892,6349892,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,6f344c45-e731-41b4-8c65-9967ebc03096,Non-Logged,3.361101
6349893,6349893,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,4c586bb4-f71d-4b39-9df8-e38ac3f632a0,Non-Logged,0.919598
6349894,6349894,5889d6ebbf62e6c115e0a280063dc8189cca490cbfea56...,855d20b7-53f2-4678-a10f-55402d085018,Non-Logged,1.990197


### Prepare data

Before fitting the LightFM model, we need to create an instance of `Dataset` which holds the interaction matrix.

In [ ]:
dataset = Dataset()

# Get unique values for users, items, and user features
unique_users = df_merged["userId"].unique()
unique_items = df_merged["history"].unique()
unique_user_features = df_merged["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=unique_user_features  # Register user features
)

In [ ]:
(interactions, weights) = dataset.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_merged.iterrows()
])

In [ ]:
user_features_list = [
    (row.userId, [row.userType])  
    for _, row in df_merged.iterrows()
]

user_features = dataset.build_user_features(user_features_list)

LightLM works slightly differently compared to other packages as it expects the train and test sets to have same dimension. Therefore the conventional train test split will not work.

The package has included the `cross_validation.random_train_test_split` method to split the interaction data and splits it into two disjoint training and test sets. 

However, note that **it does not validate the interactions in the test set to guarantee all items and users have historical interactions in the training set**. Therefore this may result into a partial cold-start problem in the test set.

In [24]:
# Split train and test sets (80/20 split)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=TEST_PERCENTAGE, random_state=SEED)
train_weights, test_weights  = cross_validation.random_train_test_split(weights, test_percentage=TEST_PERCENTAGE, random_state=SEED)


Double check the size of both the train and test sets.

In [25]:
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

Shape of train interactions: (530491, 230722)
Shape of test interactions: (530491, 230722)


In [26]:
print(f"Shape of train interactions: {train_weights.shape}")
print(f"Shape of test interactions: {test_weights.shape}")

Shape of train interactions: (530491, 230722)
Shape of test interactions: (530491, 230722)


### Fit the LightFM model

In this notebook, the LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. Further explanation on the topic can be found [here](https://making.lyst.com/lightfm/docs/examples/warp_loss.html#learning-to-rank-using-the-warp-loss).


In general, it maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present.

The LightFM model can be fitted with the following code:

In [27]:
model = LightFM(no_components=NUM_COMPONENTS,loss="warp",learning_rate=LEARNING_RATE,user_alpha=ALPHA_REG_L2,max_sampled=MAX_SAMPLED,random_state=np.random.RandomState(SEED))  # Weighted Approximate-Rank Pairwise (WARP) loss
model.fit(train, sample_weight=train_weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)


### Evaluate model

In [28]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute evaluation metrics
auc_train = auc_score(model, train, user_features=user_features, num_threads=NUM_THREADS).mean()
auc_test = auc_score(model, test, train_interactions=train, user_features=user_features, num_threads=NUM_THREADS).mean()

# Print evaluation results
print(f"AUC test Score: {auc_test:.4f}")
print(f"AUC train Score: {auc_train:.4f}")

AUC test Score: 0.8649
AUC train Score: 0.8549


### Save pkls to serve model

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
item_id_map_reverse = {v: k for k, v in item_id_map.items()}
interactions_shape = interactions.shape

In [ ]:
from utils.custom_data_structs import UserItemData

user_item_data = UserItemData(
    user_id_map = user_id_map, 
    item_id_map = item_id_map, 
    user_id_map_reverse = None, 
    item_id_map_reverse = item_id_map_reverse, 
    user_feature_map = user_feature_map, 
    item_feature_map = item_feature_map, 
    interactions_shape = interactions_shape
    )

### Import Pkls to Test Serving Model

In [ ]:
import pickle

pickle.dump(model, open('artifacts/lightfm_model.pkl', 'wb'))
pickle.dump(user_item_data, open('artifacts/user_item_data.pkl', 'wb'))

In [3]:
import pickle
from utils.custom_data_structs import UserItemData
from utils.model_funcs import recommend_by_model_scores

In [ ]:
loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_item_data:UserItemData = pickle.load(open('artifacts/user_item_data.pkl', 'rb'))

### Make predictions to known and unknowm on same recommendation function with pkls

In [6]:
# predict for known user
user_feature_list = ['userType:Logged']
user_hash = '5f5e17781fc2ec0ddcfb2e9356e61c5d3d4b0b3c8fabd20917feb9e807463856'
recommendation_list = recommend_by_model_scores(user_hash,user_feature_list,loaded_user_item_data,loaded_model)
print(recommendation_list)

['50392ead-f267-4b08-ab03-7f7c4440a8b5', 'c022aa46-74d5-4d9f-a880-6a885459d692', 'a203c57c-8693-45c8-a4bb-31f73bae4a8d', '1b31e22a-64a0-4bcd-ade2-b1bf2896c604', '25611826-eba1-49bd-b4bc-b582c9385763', 'e96714d4-00bf-4003-a27c-22fa1ccf7af4']


In [7]:
# predict for unknown user
user_feature_list = ['userType:Non-Logged']
user_hash = ''
recommendation_list = recommend_by_model_scores(user_hash,user_feature_list,loaded_user_item_data,loaded_model)
print(recommendation_list)

new user feature encountered 'userType:Non-Logged'
['d2593c3d-2347-40d9-948c-b6065e8459a9', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd', '1f32787b-de2b-49be-8c20-ddaeae34cc22', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', '4c63d7cd-4902-4ffb-9b94-578b1b2151f0']
